# Bostadsbestånd på DeSO, Stockholms län

## Inledning
Detta är ett skript för att hämta statistik över bostadsbeståndet med API från SCB för demografiska statistikområden (DeSO) inom Stockholms län.

## Hämta data från SCB
För att hämta statistik från SCB med API används biblioteket Pyscbwrapper.

Läs mer om Pyscbwrapper här: https://github.com/kirajcg/pyscbwrapper/blob/master/pyscbwrapper.ipynb

In [1]:
#Importera bibliotek
from pyscbwrapper import SCB
import pandas as pd

In [2]:
#Initiera SCB
scb = SCB('sv')

#Välj tabell
scb.go_down('BO', 'BO0104', 'BO0104X', 'BO0104T10N')

In [3]:
#Ladda hem data

import re
#Använd re för att filtrera vilken/vilka region(er) du vill hämta data för
regioner = scb.get_variables()['region']
r = re.compile('^01')
deso = list(filter(r.match, regioner))

# Skapa en fråga att skicka till databasen
scb.set_query(region = deso,
             upplåtelseform = ['hyresrätt', 'bostadsrätt', 'äganderätt', 'uppgift saknas'])

# Inspektera fråga
scb.get_query()

# Hämta metadata och extrahera data
scb_metadata = scb.get_data()
scb_data = scb_metadata['data']

# Inspektera endast de främsta raderna av data i JSON format
num_rows = 3

# Inspektera de tre första raderna
for row in scb_data[:num_rows]:
    print(row)

{'key': ['0114A0010', '1', '2015'], 'values': ['46']}
{'key': ['0114A0010', '1', '2016'], 'values': ['58']}
{'key': ['0114A0010', '1', '2017'], 'values': ['56']}


In [4]:
# Konvertera en lista av dictionaries till en dictionary med dictionaries
scb_dict = {tuple(d['key']): d['values'][0] for d in scb_data}

# Skapa en DataFrame av dictionary
df = pd.DataFrame.from_dict(scb_dict, orient='index', columns=['value'])

# Sätt ett namn för index
df.index.set_names('keys', inplace=True)
df.reset_index(inplace=True)
df[['Deso', 'upplåtelseform', 'år']] = df['keys'].apply(pd.Series)
df.drop('keys', axis=1, inplace=True)
df.rename(columns={'value':'antal_bostäder'}, inplace=True)
df['antal_bostäder'] = df['antal_bostäder'].astype(int)

# Gruppera baserat på upplåtelseform
df1 = df.groupby(['Deso', 'år'])['antal_bostäder'].sum()
df1 = pd.DataFrame(df1)
df1.reset_index(inplace=True)
df1['upplåtelseform'] = 'totalt'

# Sammanför de båda dataseten
df = pd.concat([df, df1])
df.reset_index(drop=True, inplace=True)

df.head()

,antal_bostäder,Deso,upplåtelseform,år
0,46,0114A0010,1,2015
1,58,0114A0010,1,2016
2,56,0114A0010,1,2017
3,57,0114A0010,1,2018
4,56,0114A0010,1,2019


In [5]:
#Lägg till kommunnamn som kolumn

# Definiera mappning
kommun_mapping = {
    '0114': 'Upplands Väsby',
    '0115': 'Vallentuna',
    '0117': 'Österåker',
    '0120': 'Värmdö',
    '0123': 'Järfälla',
    '0125': 'Ekerö',
    '0126': 'Huddinge',
    '0127': 'Botkyrka',
    '0128': 'Salem',
    '0136': 'Haninge',
    '0138': 'Tyresö',
    '0139': 'Upplands-Bro',
    '0140': 'Nykvarn',
    '0160': 'Täby',
    '0162': 'Danderyd',
    '0163': 'Sollentuna',
    '0180': 'Stockholm',
    '0181': 'Södertälje',
    '0182': 'Nacka',
    '0183': 'Sundbyberg',
    '0184': 'Solna',
    '0186': 'Lidingö',
    '0187': 'Vaxholm',
    '0188': 'Norrtälje',
    '0191': 'Sigtuna',
    '0192': 'Nynäshamn'
}

# Skapa 'kommun' variabel baserad på 'deso'
df['kommun'] = df['Deso'].str[:4].map(kommun_mapping)

#Ersätt värden i kolumnen upplåtelseform

# Definiera mappning
replacement_dict = {
    "1": "hyresrätt",
    "2": "bostadsrätt",
    "3": "äganderätt",
    "ÖVRIGT": "uppgift saknas"
}

# Uppdatera kolumn
df["upplåtelseform"] = df["upplåtelseform"].replace(replacement_dict)

df.head()

,antal_bostäder,Deso,upplåtelseform,år,kommun
0,46,0114A0010,hyresrätt,2015,Upplands Väsby
1,58,0114A0010,hyresrätt,2016,Upplands Väsby
2,56,0114A0010,hyresrätt,2017,Upplands Väsby
3,57,0114A0010,hyresrätt,2018,Upplands Väsby
4,56,0114A0010,hyresrätt,2019,Upplands Väsby


In [6]:
#Räkna ut andel av bostäder per upplåtelseform

# Filtrera rader där "upplåtelseform" är "totalt"
totalt_mask = df["upplåtelseform"] == "totalt"
totalt_values = df.loc[totalt_mask, ["Deso", "år", "antal_bostäder"]].rename(columns={"antal_bostäder": "totalt_value"})

# Merge totalt_values med orginal dataframe
merged_df = pd.merge(df, totalt_values, on=["Deso", "år"], how="left")

# Räkna ut "andel_av_totalt"
merged_df["andel_av_totalt"] = merged_df["antal_bostäder"] / merged_df["totalt_value"]

merged_df.head()

,antal_bostäder,Deso,upplåtelseform,år,kommun,totalt_value,andel_av_totalt
0,46,0114A0010,hyresrätt,2015,Upplands Väsby,273,0.168498
1,58,0114A0010,hyresrätt,2016,Upplands Väsby,296,0.195946
2,56,0114A0010,hyresrätt,2017,Upplands Väsby,294,0.190476
3,57,0114A0010,hyresrätt,2018,Upplands Väsby,299,0.190635
4,56,0114A0010,hyresrätt,2019,Upplands Väsby,304,0.184211
